In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [66]:
df_backend = pd.read_csv('./Data/AGU_Orders_Backend.csv')
df_returns = pd.read_csv('./Data/AGU_Returns_Backend.csv')

In [67]:
df_backend.columns

Index(['ID', 'Purchase Point', 'Purchase Date', 'Bill-to Name', 'Ship-to Name',
       'Grand Total (Base)', 'Grand Total (Purchased)', 'Status',
       'Billing Address', 'Shipping Address', 'Shipping Information',
       'Customer Email', 'Customer Group', 'Subtotal', 'Shipping and Handling',
       'Customer Name', 'Payment Method', 'Total Refunded', 'Company Name',
       'Signifyd Guarantee Decision', 'Allocated sources', 'CE Channel',
       'CE Channel Order No', 'Channable ID', 'Marketplace ID',
       'Marketplace Name', 'Marketplace Label'],
      dtype='object')

In [68]:
df_returns.columns

Index(['Credit Memo', 'Created', 'Order #', 'Order Date', 'Bill-to Name',
       'Status', 'Refunded', 'Order Status', 'Purchased From',
       'Billing Address', 'Shipping Address', 'Customer Name', 'Email',
       'Customer Group', 'Payment Method', 'Shipping Information', 'Subtotal',
       'Shipping & Handling', 'Adjustment Refund', 'Adjustment Fee',
       'Grand Total'],
      dtype='object')

In [69]:
df = df_backend.loc[:,['ID','Purchase Date','Grand Total (Purchased)','Total Refunded','Subtotal','Status','Customer Email']]

In [70]:
df = df[df.Status.isin(['processing','closed','complete'])]

In [71]:
returns = df_returns.loc[:,['Credit Memo','Created','Order #','Order Date', 'Refunded','Email']]

In [72]:
df.Status.unique()

array(['complete', 'closed', 'processing'], dtype=object)

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4590 entries, 1 to 5479
Data columns (total 7 columns):
ID                         4590 non-null int64
Purchase Date              4590 non-null object
Grand Total (Purchased)    4590 non-null float64
Total Refunded             1364 non-null float64
Subtotal                   4590 non-null float64
Status                     4590 non-null object
Customer Email             4590 non-null object
dtypes: float64(3), int64(1), object(3)
memory usage: 286.9+ KB


In [74]:
df.rename(columns={'Grand Total (Purchased)':'Revenue', 'Total Refunded':'Returns'},inplace=True)

In [75]:
df['Date'] = pd.to_datetime(pd.to_datetime(df['Purchase Date']).dt.date)
returns['Date'] = pd.to_datetime(pd.to_datetime(returns['Created']).dt.date)

In [76]:
df['Date_YM'] = df['Date'].apply(lambda x: x.strftime('%Y-%m'))

In [77]:
df.groupby(['Date_YM']).agg({'ID':pd.Series.nunique}).to_clipboard()

In [78]:
df['Year'] = df['Date'].dt.year
returns['Year'] = returns['Date'].dt.year

df['Week'] = df['Date'].dt.week
returns['Week'] = returns['Date'].dt.week

In [79]:
weekly_summary_to = df.sort_values(by=['Year','Week'],ascending=False).groupby(['Year','Week']).agg({'Revenue':pd.Series.sum, 'ID':pd.Series.nunique})
weekly_summary_returns = returns.sort_values(by=['Year','Week'],ascending=False).groupby(['Year','Week']).agg({'Refunded':pd.Series.sum, 'Order #':pd.Series.nunique})

In [80]:
weekly_summary_to.columns = ['Revenue','Transactions']

In [81]:
weekly_summary_returns.columns = ['Refunded','Refunded_Transactions']

In [82]:
weekly_summary = pd.merge(weekly_summary_to,weekly_summary_returns, on=['Year','Week'],how='left').fillna(0)

In [92]:
weekly_summary['Rev_EXVAT'] = weekly_summary['Revenue']*0.84
weekly_summary['Returns_EXVAT'] = weekly_summary['Refunded']*0.84

In [93]:
weekly_summary = weekly_summary.astype(int)

In [94]:
weekly_summary.tail(2)

Revenue  Transactions  Refunded  Refunded_Transactions  Rev_EXVAT  \
Year Week                                                                      
2020 40      90398           484     12233                     70      75934   
     41       1077             7         0                      0        904   

           Returns_EXVAT  
Year Week                 
2020 40            10275  
     41                0

In [95]:
df.sort_values(by=['Date_YM'],ascending=False).groupby(['Date_YM']).agg({'Revenue':pd.Series.sum, 'Subtotal':pd.Series.sum, 'ID':pd.Series.nunique})

,Revenue,Subtotal,ID
Date_YM,,,
2019-08,1072.72,859.29,11
2019-09,15930.15,13279.25,128
2019-10,36924.78,30572.48,229
2019-11,35390.62,30309.73,216
2019-12,28917.00,24159.17,178
2020-01,24540.22,20536.32,134
2020-02,27676.50,23020.82,175
2020-03,21225.81,17626.34,125
2020-04,30873.71,26210.44,183


In [3]:
from gspread_pandas import Spread, Client

In [6]:
agu_mt_sheet = Spread('AGU MT Weekly')

In [98]:
# Push Transactions
agu_mt_sheet.df_to_sheet(weekly_summary, index=True, sheet='DataDump_BE', start='A1', replace=True)